In [144]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pickle
from tqdm.notebook import tqdm
tqdm.pandas()

In [338]:
tweet_path = 'data/clean/twitter/full_tweet_df_v2.p'
meta_path = 'data/clean/meta/political_ads_w_metadata_v2.p'
parl_path = 'data/clean/parliament/transcripts_imputed_v2.p'

In [340]:
with open(tweet_path, 'rb') as a, open(meta_path, 'rb') as b, open(parl_path, 'rb') as c:
    tweet_df = pickle.load(a)
    meta_df = pickle.load(b)
    parl_df = pickle.load(c)

In [341]:
colmap_tw = {
        'full_text': 'doc',
        'name': 'full_name',
        'id_str': 'doc_id',
        'created_at': 'start_time',
        'description': 'tw_bio',
        'screen_name': 'tw_handle',
        'followers_count': 'tw_followers'
    }

colmap_meta = {
        'ad_creative_bodies': 'doc',
        'id': 'doc_id',
        'bylines': 'meta_sponsor',
        'ad_delivery_start_time': 'start_time',
        'ad_delivery_stop_time': 'end_time',
        'page_name': 'full_name',
        'name': 'wiki_name',
        'ad_snapshot_url': 'meta_url',
        'demographic_distribution': 'meta_demography',
        'delivery_by_region': 'meta_region',
        'estimated_audience_size': 'meta_audience',
        'publisher_platforms': 'meta_platforms',
        'spend': 'meta_spend',
        'currency': 'meta_currency',
        'impressions': 'meta_impressions',
        'ad_creative_link_titles': 'meta_titles',
        'ad_creative_link_captions': 'meta_captions',
        'ad_creative_link_descriptions': 'meta_descriptions',
        'birth_year': 'wiki_birth_year',
        'group_name': 'party',
        'electoral_region': 'wiki_electoral_region',
        'election_data_year': 'wiki_election_year',
        'education': 'wiki_edu',
        'personal_votes': 'wiki_votes'
    }

colmap_parl = {
        'text': 'doc',
        'group_name': 'party',
        'role': 'parl_role'
    }

In [342]:
tweet_df['source'] = 'twitter'
tweet_df = tweet_df.rename(
    columns = colmap_tw
)
tweet_df.head(2)

,tw_handle,full_name,tw_bio,tw_followers,doc_id,start_time,doc,source
0,AndersKuhnau,Anders Kühnau,Regionsrådsformand i Region Midtjylland. Forma...,1864,1570289445332459523,2022-09-15 05:52:32+00:00,@HrTousgaard @JTzsche @regionerne @Sygeplejera...,twitter
1,AndersKuhnau,Anders Kühnau,Regionsrådsformand i Region Midtjylland. Forma...,1864,1570069366816792576,2022-09-14 15:18:02+00:00,@JTzsche @regionerne @Sygeplejeraadet Du ved f...,twitter


In [343]:
meta_df['source'] = 'meta'
meta_df = meta_df.rename(
    columns = colmap_meta
)
meta_df.head(2)

,page_id,full_name,meta_sponsor,doc,start_time,meta_url,meta_demography,meta_region,meta_audience,meta_platforms,...,meta_descriptions,level,wiki_name,wiki_birth_year,party,wiki_electoral_region,wiki_election_year,wiki_edu,wiki_votes,source
0,132364993978468,Andreas Steenberg,Andreas Steenberg,[Hvor dum og uretfærdig kan vores udlændingelo...,2022-10-22,https://www.facebook.com/ads/archive/render_ad...,"[{'percentage': '0.026316', 'age': '55-64', 'g...","[{'percentage': '1', 'region': 'Central Denmar...","{'lower_bound': '100001', 'upper_bound': '5000...","[facebook, instagram]",...,NaN,politician,Andreas Steenberg,1983 (35 år),Radikale Venstre,Vestjylland (Kredsmandat),2019.0,cand.scient.pol.,4.444,meta
1,132364993978468,Andreas Steenberg,Andreas Steenberg,[Hvor dum og uretfærdig kan vores udlændingelo...,2022-10-22,https://www.facebook.com/ads/archive/render_ad...,"[{'percentage': '0.065789', 'age': '45-54', 'g...","[{'percentage': '0.230263', 'region': 'Central...","{'lower_bound': '50001', 'upper_bound': '100000'}","[facebook, instagram]",...,NaN,politician,Andreas Steenberg,1983 (35 år),Radikale Venstre,Vestjylland (Kredsmandat),2019.0,cand.scient.pol.,4.444,meta


In [344]:
parl_df['source'] = 'parliament'
parl_df = parl_df.rename(
    columns = colmap_parl
)
parl_df.head(2)

,first_name,last_name,full_name,party,parl_role,doc,year,day,time,start_time,end_time,duration,source
0,Lars Løkke,Rasmussen,Lars Løkke Rasmussen,"V,UFG,M",minister,Danmark er et godt land at leve i. Vi har bygg...,2009,10-06,12:04:01,2009-10-06T12:04:01,2009-10-06T12:50:39,0 days 00:46:38,parliament
1,Bjarne,Laustsen,Bjarne Laustsen,S,medlem,"Tak, hr. formand. Den sag, jeg vil bringe på b...",2009,10-07,13:04:25,2009-10-07T13:04:25,2009-10-07T13:06:31,0 days 00:02:06,parliament


In [345]:
df_concat = pd.concat([tweet_df, meta_df, parl_df])

In [346]:
df_concat.columns

Index(['tw_handle', 'full_name', 'tw_bio', 'tw_followers', 'doc_id',
       'start_time', 'doc', 'source', 'page_id', 'meta_sponsor', 'meta_url',
       'meta_demography', 'meta_region', 'meta_audience', 'meta_platforms',
       'meta_spend', 'meta_currency', 'meta_impressions', 'meta_titles',
       'end_time', 'meta_captions', 'meta_descriptions', 'level', 'wiki_name',
       'wiki_birth_year', 'party', 'wiki_electoral_region',
       'wiki_election_year', 'wiki_edu', 'wiki_votes', 'first_name',
       'last_name', 'parl_role', 'year', 'day', 'time', 'duration'],
      dtype='object')

In [347]:
#(
#    df_concat[['source', 'party', 'full_name']]
#        .sort_values(by = 'full_name')
#        .drop_duplicates(subset=['full_name'])
#).to_excel('politician_names.xlsx')

In [348]:
mapped_names = pd.read_excel('politician_names_coded.xlsx')

In [349]:
mapped_names = mapped_names.loc[mapped_names['mapping'].notna(), ['full_name', 'mapping']].set_index('full_name')

In [350]:
mapped_names

,mapping
full_name,
Aaja Chemnitz Larsen,Aaja Chemnitz
"Aaja Chemnitz Larsen, Inuit Ataqatigiit",Aaja Chemnitz
"Aaja Chemnitz, Inuit Ataqatigiit",Aaja Chemnitz
Aki Høegh-Dam,Aki-Matilda Høegh-Dam
Ane Halsboe-Larsen,Ane Halsboe-Jørgensen
...,...
karen lagoni,Karen Lagoni
malene busk,Malene Busk
melanie simick,Melanie Simick


In [351]:
df_concat['full_name'] = df_concat['full_name'].map(mapped_names['mapping']).fillna(df_concat['full_name'])

In [352]:
df = df_concat.copy()

In [353]:
df['party'].value_counts()

S                              62774
V                              53013
DF                             40235
EL                             34710
SF                             25689
KF                             21412
RV                             20403
LA                             18167
ALT                             8670
NB                              5164
Venstre                         4651
Socialistisk Folkeparti         2905
V,UFG                           2348
UFG                             1875
V,UFG,M                         1726
Det Konservative Folkeparti     1718
Socialdemokratiet               1701
SF,S                            1402
Nye Borgerlige                  1361
Radikale Venstre                1240
IA                              1223
Liberal Alliance                1036
KD                               813
Dansk Folkeparti                 738
FG                               643
Enhedslisten                     557
SIU                              507
J

In [354]:
party_map = {
    'Radikale Venstre': 'RV',
    'Venstre': 'V',
    'Socialistisk Folkeparti': 'SF',
    'Det Konservative Folkeparti': 'KF',
    'Socialdemokratiet': 'S',
    'Nye Borgerlige': 'NB',
    'Liberal Alliance': 'LA',
    'Dansk Folkeparti': 'DF',
    'Enhedslisten': 'EL',
    'Alternativet': 'ALT',
    'Inuit Ataqatigiit': 'IA',
    'Siumut': 'SIU'
}

In [355]:
for old, new in tqdm(party_map.items()):
    df['party'] = df['party'].str.replace(old, new, regex=False)

  0%|          | 0/12 [00:00<?, ?it/s]

In [356]:
df['party'].value_counts()

S                64475
V                57664
DF               40973
EL               35267
SF               28594
KF               23130
RV               21643
LA               19203
ALT               8758
NB                6525
V,UFG             2348
UFG               1875
V,UFG,M           1726
SF,S              1402
IA                1290
KD                 813
FG                 643
SIU                510
JF                 505
T                  287
SP                 286
SF,RV,S            190
LA,UFG             139
DD                 106
RV,UFG,ALT,FG      102
NQ                  97
TF                  65
M                   37
Name: party, dtype: int64

In [357]:
df.shape

(1030988, 37)

In [359]:
with open('data/clean/combined/df_combined.p', 'wb') as p:
    pickle.dump(df, p)

In [304]:
# TODO: Impute parties based on last known value for each individual politician
# Set party for each party page based on mapping
# Set type for party pages on twitter; rest is "politician"
# Filter out any non-MPs based on wiki list
# Add labels for political leaders

In [276]:
df.groupby('full_name').filter(lambda x: x[['party']].isnull().all().all())

,tw_handle,full_name,tw_bio,tw_followers,doc_id,start_time,doc,source,page_id,meta_sponsor,...,wiki_election_year,wiki_edu,wiki_votes,first_name,last_name,parl_role,year,day,time,duration
0,AndersKuhnau,Anders Kühnau,Regionsrådsformand i Region Midtjylland. Forma...,1864.0,1570289445332459523,2022-09-15 05:52:32+00:00,@HrTousgaard @JTzsche @regionerne @Sygeplejera...,twitter,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AndersKuhnau,Anders Kühnau,Regionsrådsformand i Region Midtjylland. Forma...,1864.0,1570069366816792576,2022-09-14 15:18:02+00:00,@JTzsche @regionerne @Sygeplejeraadet Du ved f...,twitter,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AndersKuhnau,Anders Kühnau,Regionsrådsformand i Region Midtjylland. Forma...,1864.0,1555303386333020164,2022-08-04 21:23:18+00:00,RT @larsgaardhoj: Barsel med mine børn er noge...,twitter,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AndersKuhnau,Anders Kühnau,Regionsrådsformand i Region Midtjylland. Forma...,1864.0,1555303283874566147,2022-08-04 21:22:53+00:00,RT @LarsSandahlS: Fra i dag går fædre fra at h...,twitter,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AndersKuhnau,Anders Kühnau,Regionsrådsformand i Region Midtjylland. Forma...,1864.0,1544631984386883586,2022-07-06 10:38:57+00:00,RT @regionerne: KRONIK FOR MILJØET: Danmark ka...,twitter,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26834,NaN,SF,NaN,NaN,366735390851431,2018-05-17,[Den danske natur er fantastisk! Men vi skal h...,meta,74796954245,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26835,NaN,SF,NaN,NaN,309108026436856,2018-05-17,[Den danske natur er fantastisk. Men vi skal h...,meta,74796954245,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26836,NaN,SF,NaN,NaN,2097782130298398,2018-05-17,[Den danske natur er fantastisk! Men vi skal h...,meta,74796954245,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26837,NaN,SF,NaN,NaN,404462130101559,2018-05-17,[Den danske natur er fantastisk. Men vi skal h...,meta,74796954245,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [253]:
df.sort_values(by = ['full_name', 'party'])
df['party'] = df.groupby('full_name')['party'].transform(lambda v: v.ffill())

In [254]:
df[df['party'].isna()].shape

(680243, 37)

In [281]:
list(df.columns)

['tw_handle',
 'full_name',
 'tw_bio',
 'tw_followers',
 'doc_id',
 'start_time',
 'doc',
 'source',
 'page_id',
 'meta_sponsor',
 'meta_url',
 'meta_demography',
 'meta_region',
 'meta_audience',
 'meta_platforms',
 'meta_spend',
 'meta_currency',
 'meta_impressions',
 'meta_titles',
 'end_time',
 'meta_captions',
 'meta_descriptions',
 'level',
 'wiki_name',
 'wiki_birth_year',
 'party',
 'wiki_electoral_region',
 'wiki_election_year',
 'wiki_edu',
 'wiki_votes',
 'first_name',
 'last_name',
 'parl_role',
 'year',
 'day',
 'time',
 'duration']

In [287]:
mps = pd.read_csv('data/raw/parliament/MP_names_15_19.csv')

In [290]:
mp_names = mps['name']

In [296]:
type(df['start_time'].iloc[0])

pandas._libs.tslibs.timestamps.Timestamp

In [300]:
df['start_time'] = pd.to_datetime(df['start_time'], utc=True)

In [334]:
mps.loc[mps['name'] == 'Ane Halsboe-Jørgensen']

,name,birth_year,group_name,electoral_region,election_data_year,education,personal_votes,page_id
8,Ane Halsboe-Jørgensen,1983 (36 år),Socialdemokratiet,Nordjylland (Tillægsmandat),2019,cand.scient.pol.,4.977,1.699388e+14


In [333]:
df.loc[
    df['full_name'].isin(mp_names) &
    (df['start_time'] < '2021-07-05 00:00:00') &
    (df['source'] == 'meta'),
    [
    'doc', 'doc_id', 'source', 'start_time',
    'full_name', 'tw_handle', 'level', 'party',
    'parl_role', 'wiki_edu', 'wiki_votes'
    ]
].drop_duplicates(subset='full_name').head(50)

,doc,doc_id,source,start_time,full_name,tw_handle,level,party,parl_role,wiki_edu,wiki_votes
666,[Hvad skal det nytte?\n\nKom til et spændende ...,243502120577571,meta,2021-07-02 00:00:00+00:00,Andreas Steenberg,NaN,politician,RV,NaN,cand.scient.pol.,4.444
667,"[Du skal da følge med i, hvad jeg render og la...",581252149527721,meta,2021-06-28 00:00:00+00:00,Alex Vanopslagh,NaN,politician,LA,NaN,cand.scient.pol.,3.337
723,[Naggasiinermi oqalugiaatiga nutaarsiassaniitt...,319941532951393,meta,2021-06-10 00:00:00+00:00,Aki-Matilda Høegh-Dam,NaN,politician,SIU,NaN,BA (scient.pol.,3.467
786,[‼️Ministerens nølen og manglende vilje til at...,462714094851730,meta,2021-02-18 00:00:00+00:00,Anne Honoré Østergaard,NaN,politician,V,NaN,cand.merc.jur.,4.104
965,[Jeg havde her sidst på eftermiddagen fornøjel...,681132865712751,meta,2019-10-28 00:00:00+00:00,Anders Kronborg,NaN,politician,S,NaN,cand.polit.,8.607
1025,[Danmark skal have en ny regering! Stem person...,883809418623903,meta,2019-06-05 00:00:00+00:00,Ane Halsboe-Jørgensen,NaN,politician,S,NaN,cand.scient.pol.,4.977
1045,[Stem personligt på Anne Paulin (S) for et grø...,309802713272359,meta,2019-06-03 00:00:00+00:00,Anne Paulin,NaN,politician,S,NaN,M.Sc. (international business and politics),6.814
1783,[Kampen er slut - ådalen og heden er reddet 💚\...,908737483016845,meta,2021-06-28 00:00:00+00:00,Birgitte Vind,NaN,politician,S,NaN,lærer,7.012
1784,[Sådan en vil jeg gerne bygge fra Silkeborg ov...,209617007689833,meta,2021-06-27 00:00:00+00:00,Anne Valentina Berthelsen,NaN,politician,SF,NaN,BA (politik og administration),1.304
1786,[Ny videregående uddannelse kommer til Helsing...,803594530299530,meta,2021-06-25 00:00:00+00:00,Birgitte Bergman,NaN,politician,KF,NaN,BA (business administration),2.524


In [284]:
df[[
    'doc', 'doc_id', 'source', 'start_time',
    'full_name', 'tw_handle', 'level', 'party',
    'parl_role', 'wiki_edu', 'wiki_votes'
    ]].drop_duplicates(subset = 'full_name').head(50)

,doc,doc_id,source,start_time,full_name,tw_handle,level,party,parl_role,wiki_edu,wiki_votes
0,@HrTousgaard @JTzsche @regionerne @Sygeplejera...,1570289445332459523,twitter,2022-09-15 05:52:32+00:00,Anders Kühnau,AndersKuhnau,NaN,NaN,NaN,NaN,NaN
511,@madskastrup Vidste I at de nye længe ventede ...,1561127883095121921,twitter,2022-08-20 23:07:46+00:00,Simon Hansen,stemsimonhansen,NaN,NaN,NaN,NaN,NaN
1290,@lineervolder Hvad er det interessante? I blå ...,1571732892220362752,twitter,2022-09-19 05:28:17+00:00,Jonas Bjørn Jensen,Jonasbjorn,NaN,NaN,NaN,NaN,NaN
3919,RT @DMCamilla: Se interessant🧵om kønsulighed a...,1571794541656608773,twitter,2022-09-19 09:33:15+00:00,Kvinderådet,Kvinderaadet,NaN,NaN,NaN,NaN,NaN
6504,"Presseinvitation: Kom med, når Miljøministerie...",1570711236177973248,twitter,2022-09-16 09:48:35+00:00,Miljøministeriet,miljoemin,NaN,NaN,NaN,NaN,NaN
9685,@TheTweetOfGod Lol,1567808653368999938,twitter,2022-09-08 09:34:46+00:00,Tommy Petersen,TommyPetersenDK,NaN,NaN,NaN,NaN,NaN
12885,@SamFazeli8 Monkeypox - then Bavarian Nordic h...,1527404394416852996,twitter,2022-05-19 21:42:40+00:00,PGraversgaard,PGraversgaard,NaN,NaN,NaN,NaN,NaN
12923,"Via ""Omprioritering"" vil DF og V lade Gentofte...",731094522088804352,twitter,2016-05-13 12:11:31+00:00,Hans Toft,htgentofte,NaN,NaN,NaN,NaN,NaN
12928,@MichaelKrasnik Ja det er trist læsning.,1460299475487014917,twitter,2021-11-15 17:31:40+00:00,Christel Gall,CgaChristel,NaN,NaN,NaN,NaN,NaN
13180,"Vi skal kunne smage på, fornemme og relatere t...",1549522253058121728,twitter,2022-07-19 22:31:08+00:00,Sascha Haunstrup Qvortrup,perleberger,NaN,NaN,NaN,NaN,NaN


In [243]:
df[df['party'].isna()].shape

(680243, 37)

In [244]:
df = df.sort_values(by='full_name')
df['party'] = df['party'].ffill()

In [245]:
df[df['party'].isna()].shape

(0, 37)

In [231]:
df_test.loc[df_test['party'].isna(), ['full_name', 'party', 'part']].tail(7000)

,full_name,party,part
78368,Winni Grosbøll,NaN,KF
78369,Winni Grosbøll,NaN,KF
78370,Winni Grosbøll,NaN,KF
78371,Winni Grosbøll,NaN,KF
78372,Winni Grosbøll,NaN,KF
...,...,...,...
16792,Özkan Kocak,NaN,Radikale Venstre
16793,Özkan Kocak,NaN,Radikale Venstre
16794,Özkan Kocak,NaN,Radikale Venstre
16795,Özkan Kocak,NaN,Radikale Venstre


____

## FuzzyJoin

In [7]:
with open('data/clean/twitter/full_tweet_df_19-09-22.p', 'rb') as p:
    tweet_df = pickle.load(p)

In [13]:
ad_files = ['politician_ads_v1']#, 'party_ads_v1']
ad_dfs = []

for ad_file in ad_files:
    with open(f'data/clean/meta/{ad_file}.p', 'rb') as p:
        ad_dfs.append(pickle.load(p))

ad_df = pd.concat(ad_dfs).reset_index(drop = True)

In [15]:
print(ad_df['page_name'].head())
print(tweet_df['screen_name'].head())

0    Andreas Steenberg
1    Andreas Steenberg
2    Andreas Steenberg
3    Andreas Steenberg
4    Andreas Steenberg
Name: page_name, dtype: object
0    AndersKuhnau
1    AndersKuhnau
2    AndersKuhnau
3    AndersKuhnau
4    AndersKuhnau
Name: screen_name, dtype: object


In [57]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=75, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].progress_apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].progress_apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1

In [58]:
twitter_names = tweet_df[['screen_name']].drop_duplicates()
meta_names = ad_df[['page_name']].drop_duplicates()

In [59]:
fuzzy_test = fuzzy_merge(twitter_names, meta_names, 'screen_name', 'page_name')

  0%|          | 0/638 [00:00<?, ?it/s]

  0%|          | 0/638 [00:00<?, ?it/s]

In [61]:
fuzzy_test.loc[fuzzy_test['matches'] != ''].tail(50)

,screen_name,matches
4032,simonkollerup,Simon Kollerup
7369,stinuslindgreen,Stinus Lindgreen - Radikale Venstre
10569,KirstenNormann,Kirsten Normann Andersen
27849,MadsAndersenC,Hans Andersen
28017,SorenPape,Søren Pape Poulsen
30838,SorenEggeRasmus,Søren Egge Rasmussen
33595,RasmusPrehn,Rasmus Prehn
36796,MarieKrarup,Marie Krarup
39173,AlexVanopslagh,Alex Vanopslagh
51719,Kristianthdahl,Kristian Thulesen Dahl


In [5]:
fuzz.ratio('Søren Pape Poulsen', 'Søren Pape')

71

In [ ]:
p